## 2020빅콘테스트 - 챔스리그 

### NS SHOP+ 판매실적 예측을 통한 편성 최적화 방안(모형) 도출

NS Shop+편성데이터(NS홈쇼핑) 를 활용하여 방송편성표에 따른 판매실적을 예측하고, 최적 수익을 고려한 요일별/ 시간대별 / 카테고리별 편성, 최적화 방안(모형) 제시



In [1]:
import struct
print(struct.calcsize("P") * 8) 
# 파이썬 몇bit인지 확인할 수 있다. 

64


In [2]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from datetime import datetime 
from datetime import timedelta

In [3]:
# !pip install tensorflow
# !pip install keras

In [4]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [5]:
sales = pd.read_excel("C:/Users/jenny/Desktop/빅콘/2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그)/01_제공데이터/sales.xlsx",header= 1)
tvrates = pd.read_excel("C:/Users/jenny/Desktop/빅콘/2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그)/01_제공데이터/tvrates.xlsx", header = 1)
test = pd.read_excel("C:/Users/jenny/Desktop/빅콘/2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그)/02_평가데이터/test.xlsx", header =1)

In [6]:
sales.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


pandas-profiling, 추후 특정 모델 한글 column 표시 안되므로 영문으로 변경

In [7]:
test.columns=['datetime','duration','code_mother','code_product','product','category','price','totprice']
sales.columns=['datetime','duration','code_mother','code_product','product','category','price','totprice']

In [8]:
d = pd.Categorical(sales['category'], categories = ['주방','가전','의류','속옷','잡화','농수축','생활용품','가구','이미용','무형','건강기능','침구']) 
d.categories = ["kitchen","homeAppliance","clothes","underwear","item","agri","dailyitem","furniture","beauty","intangible","health","bedding"]
sales['category']=d

In [9]:
# display(tvrates, sales)

In [10]:
display(sales.dtypes, sales.shape, tvrates.dtypes, tvrates.shape )

datetime        datetime64[ns]
duration               float64
code_mother              int64
code_product             int64
product                 object
category              category
price                    int64
totprice               float64
dtype: object

(38309, 8)

시간대                          object
2019-01-01                  float64
2019-01-02                  float64
2019-01-03                  float64
2019-01-04                  float64
                             ...   
2019-12-28                  float64
2019-12-29                  float64
2019-12-30                  float64
2019-12-31                  float64
2019-01-01 to 2019-12-31    float64
Length: 367, dtype: object

(1441, 367)

### 결측치 처리

판매실적(sales) 데이터에서 동시 방송(노출)된 상품의 경우, 데이터상의 '노출(분)'이 생략되어 있다. 추후 시청률 데이터를 '방송일시' 및 '노출(분)'과의 연결을 위해 0으로 처리한다. 

In [11]:
sales['duration'] =sales['duration'].fillna(0) 
# 좌측을 sales라고만 하면 안돼! 

In [12]:
# pd.DataFrame(sales)
# # 결측치가 정상적으로 처리됨을 확인할 수 있다. 

### 변수 제거 및 생성


#### 1-(1) sales row 제거
무형 상품의 경우, 예측 불필요 > 제거 > 새로운 데이터셋인 sales1으로 저장

In [13]:
sales = sales[sales['category'] != 'intangible']

#### 1-(2) tvrates 마지막 row/col 제거 
마지막 row는 각 일자별 분당 시청률 평균에 해당하므로 제거
마지막 col은 각 시간별 분당 시청률 평균에 해당하므로 제거

In [14]:
tvrates = tvrates.drop(1440)

In [15]:
tvrates = tvrates.drop(columns = "2019-01-01 to 2019-12-31")

In [16]:
# tvrates


#### 2-(1) '방송일시'에서 가능한 변수들 우선 모두 추가하기. (나중에 중요도로 선택적 결정)

In [17]:
sales['datetime'] = sales['datetime'].astype('datetime64')
sales['weekday'] = sales['datetime'].dt.weekday # 0: Sunday, 6: Sat
sales['time'] =  sales['datetime'].dt.time
sales['month'] =  sales['datetime'].dt.month
sales['date'] = sales['datetime'].dt.date       # 연, 월, 일

#### 2-(2) 판매실적(sales) 데이터에 판매수량 column 추가 


일반적 수식인 취급액 = 판매단가X주문량 사용. 이때, 소숫점은 다양한 할인/쿠폰 등의 사용으로 발생할 수 있음

In [18]:
sales["quantitiy"] = round(sales["totprice"]/sales["price"])

#### 2-(3) tvrates의 각 방송시간별 시청률 평균만 sales에 추가하기 

우선은 20분 간격이라고 가정
<br>
20분 단위로 시청률 묶어볼 것
<br>
노출시간이 20분이라고 하였을 때, 다음과 같이 전개가능하다. 

In [19]:
# tvrates

In [20]:
# index = []
# for i in range(72):
#     index += [i]*20

In [21]:
# # index꼴 확인하기
# index

In [22]:
# tvrates['index'] = index

In [23]:
# tvrates_temp = tvrates.drop('시간대', 1 )

In [24]:
# tvrates_temp.groupby('index').mean()

### 그럼 이제 주어진 노출시간 이용하기  

주어진 duration(노출시간)이 float64 > datetime과 바로 연산 불가

In [25]:
sales.dtypes

datetime        datetime64[ns]
duration               float64
code_mother              int64
code_product             int64
product                 object
category              category
price                    int64
totprice               float64
weekday                  int64
time                    object
month                    int64
date                    object
quantitiy              float64
dtype: object

In [26]:
# datetime > 절대량 timestamp > 수치화
# timestamp > duration 분 더하기 ?

In [27]:
sales['dur_min'] = pd.to_datetime(sales['duration'], format='%M').dt.time

In [28]:
sales.dtypes

datetime        datetime64[ns]
duration               float64
code_mother              int64
code_product             int64
product                 object
category              category
price                    int64
totprice               float64
weekday                  int64
time                    object
month                    int64
date                    object
quantitiy              float64
dur_min                 object
dtype: object

In [29]:
sales['dur_min']

0        00:20:00
1        00:00:00
2        00:20:00
3        00:00:00
4        00:20:00
           ...   
38299    00:00:00
38300    00:20:00
38301    00:00:00
38302    00:00:00
38303    00:00:00
Name: dur_min, Length: 37372, dtype: object

In [30]:
# sales 행렬 크기 
sales.shape

(37372, 14)

In [31]:
# sales['datetime'] + sales['dur_min']

In [32]:
# sd = sales["duration"]
# givendate = sales["datetime"]

In [33]:
# givendate

In [34]:
# print( )

In [35]:
# display(sd[1], givendate[1])

In [36]:
# sales.shape[0] 
# # 37372 중 무형빼고 37372 남았다. 

In [37]:
# for i in range(37372):
#     print(givendate[i])

In [38]:
# for i in range(sales.shape[0]):
#     print(sd[i])

In [39]:
# for i in range(sales.shape[0]):
#     print(givendate[i] + timedelta(minutes = sd[i]))

In [40]:
# sales['datetime'] 

In [41]:
# pd.DatetimeIndex(sales['datetime'])
# pd.DatetimeIndex(sales['duration'])

### Pandas Profiling 
(영훈) EDA이전 대략적인 추이 보기 좋음

In [42]:
# pip install pandas-profiling

In [43]:
# pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [44]:
# import pandas_profiling as pp
# pp.ProfileReport(sales)

In [45]:
# sales.groupby(['code_mother', 'product'])['datetime'].unique().reset_index()

### NLP

#### 혹시 Komoran?
역시나 사용불가 : 브랜드 및 상품명 인식 불가 

In [46]:
# !pip install konlpy

In [47]:
# from konlpy.tag import Komoran
# komoran = Komoran() 
# komoran.morphs('오늘너무피곤해서지각을할뻔했습니다')

# sales['kormoran_text'] = sales['product'].apply(lambda x:komoran.morphs(x))
# sales


#### 상품명 빈도수를 알아보자. 
주어진 데이터의 상품명은 띄어쓰기가 잘 되어있는 편이다. > Tokenizer
<br/>

##### Q.  tk.word_counts를 갯수 오름차순으로 보는 법 ,,,알고 싶다 

In [ ]:
from keras.preprocessing.text import Tokenizer 
tk = Tokenizer()
tk.fit_on_texts(list(sales['product']))

In [ ]:
tk.word_counts

In [ ]:
tk.word_index

In [ ]:
howmuchaday = pd.DataFrame(sales.groupby('datetime').sum()['totprice'])

In [ ]:
pd.DataFrame(howmuchaday['totprice'])

In [ ]:
datetime','duration','code_mother','code_product','product','category','price','totprice']

In [ ]:
sales.groupby('code_product')['price'].nunique().reset_index()

In [ ]:
sales.nunique()

In [ ]:
temp1 = sales.groupby(['code_mother','code_product'])['product'].unique().reset_index()
temp1
# 마더코드-상품코드 2123개

In [ ]:
a = sales.groupby('category').sum()['totprice'].reset_index()
a

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize = (20,12))
# sns.boxplot(alldata['outcome'], alldata['bid_id'], showfliers = False) 

In [ ]:
# plt.figure(figsize = (20,12))
# sns.boxplot(sales, hue = a['totprice'], showfliers = False) 

In [ ]:
sales

In [ ]:
sales["wknd"] 
if sales['weekday'] < 5 print("0") else print("1") 

In [ ]:
from datetime import date
from workalendar.asia import SouthKorea

In [ ]:
sales.head(20)